In [ ]:
import marimo as mo

#Kasus 4 - Operasi Group dan Wawasan
- Data Engineering
- Ludy Hasby Aulia : 2702409305

##Load Library

In [ ]:
import pandas as pd
import numpy as np

##Dataset

In [ ]:
np.random.seed(42) 

In [ ]:
n = 1000 
transactions = pd.DataFrame({ 
    "transaction_id": [f"T{i:04d}" for i in range(1, n+1)], 
    "seller_id": np.random.choice(["S001", "S002", "S003", "S004", "S005"], n), 
    "category": np.random.choice(["Electronics", "Fashion", "Food", "Home", "Beauty"], n), 
    "amount": np.random.randint(50000, 1000000, n), 
    "quantity": np.random.randint(1, 10, n), 
    "payment_method": np.random.choice(["Credit Card", "E-Wallet", "Bank Transfer", "COD"], n), 
    "city": np.random.choice(["Jakarta", "Bandung", "Surabaya", "Medan", "Makassar"], n), 
    "date": pd.date_range("2025-01-01", periods=n), 
    "status": np.random.choice(["completed", "cancelled", "pending"], n, p=[0.7, 0.2, 0.1]) 
}) 

In [ ]:
transactions

##Group Dasar dan Wawasan

###Penjualan setiap Penjual

In [ ]:
# hitung pendapatan per penjual
revenuePerSeller = transactions.groupby("seller_id")["amount"].sum().sort_values(ascending=False).reset_index()
revenuePerSeller

###Persentase setiap Penjual terhadap total

In [ ]:
totalRevenue = transactions['amount'].sum()

revenuePerSeller['percentageP'] = revenuePerSeller['amount']/totalRevenue

In [ ]:
revenuePerSeller

###Rata-rata Jumlah Transaksi per Kategori

In [ ]:
meanCategory = transactions.groupby("category")["quantity"].mean().sort_values(ascending=False)
meanCategory

##Group Multilevel

###Total Pendapatan berdasar seller_id dan category

In [ ]:
sellerCategory = transactions.groupby(["seller_id", "category"])["amount"].sum().reset_index().sort_values('amount', ascending=False)
sellerCategory

###Pivot Tabel Seller - Category

In [ ]:
pivotSellerCategory = transactions.pivot_table(
    index="seller_id", 
    columns="category", 
    values="amount", 
    aggfunc="sum"
)

pivotSellerCategory

###Kategori terbaik setiap seller

In [ ]:
topCategoryPerSeller = sellerCategory.drop_duplicates(subset=['seller_id'], keep='first')
print(topCategoryPerSeller)

##Agregasi Kustom

In [ ]:
def mean_transaksi_complete(x):
    mask = transactions.loc[x.index, "status"] == "completed"
    return x[mask].mean()

In [ ]:
def total_pendapatan_complete(x):
    mask = transactions.loc[x.index, "status"] == "completed"
    return x[mask].sum()

In [ ]:
def tingkat_penyelesaian(x):
    return (x == "completed").sum() / len(x)

In [ ]:
def get_mode(x):
    return x.mode().iloc[0] if not x.mode().empty else None

In [ ]:
customeAgg = transactions.groupby('seller_id').agg(
    jumlah_transaksi=('transaction_id', 'count'),
    total_pendapatan_complete=('amount', total_pendapatan_complete), 
    mean_transaksi_complete=('quantity', mean_transaksi_complete), 
    tingkat_penyelesaian=('status', tingkat_penyelesaian), 
    metode_bayar_umum=('payment_method', get_mode)   
)

customeAgg

In [ ]:
# tambahkan peringkat 
customeAgg['rank_pendapatan'] = customeAgg['total_pendapatan_complete'].rank(ascending=False, method='dense').astype(int)

customeAgg

In [ ]:
# penjual yang layak diberi reward 
customeAgg.sort_values(["total_pendapatan_complete", "tingkat_penyelesaian"], ascending=False)

In [ ]:
# penjual perlu diperhatikan 
customeAgg.sort_values(["tingkat_penyelesaian", "total_pendapatan_complete"], ascending=True)